In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv("C:/Users/rithw/Desktop/Udemy/UPDATED_NLP_COURSE/05-Topic-Modeling/npr.csv")

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
npr.shape

(11992, 1)

In [5]:
npr['Article'][10]

'For years now, some of the best, wildest, most moving or revealing stories we’ve been telling ourselves have come not from books, movies or TV, but from video games. So we’re running an occasional series, Reading The Game, in which we take a look at some of these games from a literary perspective. I played the game through the first time in something like a perfect state of awe and terror. Enraptured is, I think, the word that best describes it. Carried away completely into this ruined, beautiful world and the story of Joel and Ellie in The Last of Us. Normally such a completionist  —   so obsessed with exploring every hide and hollow in these imaginary worlds I throw myself into  —   in this instance I simply rolled with the narrative. Ran when running was proper. Slogged through dark and rain and snow and sunshine. Stood my bloody ground when left with no other options. Joel came to love Ellie, his surrogate daughter, and Ellie came to love Joel, the only father she’d ever known. An

In [6]:
npr['Article'][1000]

'The Fred Hersch Trio brings a seductive and crafty intelligence to its version of ”We See,” the Thelonious Monk tune. Articulating its melody at the piano, Hersch slips in a few leisurely pauses, which slow down and stretch out the form. Then, in the bridge, he ratchets up to twice the speed, evoking the frenetic whir of the factory machinery in Charlie Chaplin’s Modern Times. The track suggests a clever, affectionate tribute to an enduring work of art  —   and that’s before Hersch even gets around to his jaunty yet exploratory piano solo. On Sunday, Hersch could conceivably win a Grammy for that solo, and another for Sunday Night at the Vanguard, the album on which ”We See” appears. Hersch is a veteran Grammy nominee, and he’d be a deserving winner. But along with a high level of artistry, his nominations this year attest to a persistent and pervasive truth in jazz: When it comes to glittery recognition from the Recording Academy, as with broader popular acclaim, the scale is usually

# Pre-processing the data

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(min_df=2, max_df=0.95, stop_words="english", lowercase=True)


    min_df = Minimum number of documents or minuimum % of documents that word has to appear in to be counted i.e. remove rare words
    
    max_df = percentage of documents that word has to appear in in order to not count those word. i.e. Remove extremely common words 

    lowercase = True : converting all the documents to lowercase
    
    stop_words = english : removing the stop words


In [19]:
dtm = cv.fit_transform(npr['Article'])

In [20]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

11992x54777 : no of documents X no of words

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
lda = LatentDirichletAllocation(n_components = 10)

    n_components : Deciding on the number of topics


In [23]:
lda.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

After fitting LDA, :
    
    1. Grab the vocabulary of words
    
    2. Grab the topics
    
    3. Grab the highest probability words in each topic

### Vocabulary of Words

In [24]:
len(cv.get_feature_names( ))

54777

In [26]:
type(cv.get_feature_names())

list

In [31]:
cv.get_feature_names()[12382]

'curriculum'

In [32]:
cv.get_feature_names()[15382]

'dripping'

In [35]:
import random

cv.get_feature_names()[random.randint(0,54777)]

'tulum'

### The Topics

In [36]:
lda.components_

array([[2.12175668e+01, 1.09715806e+03, 1.00000000e-01, ...,
        1.00002856e-01, 1.00064876e-01, 1.00000000e-01],
       [1.90143076e-01, 5.63081073e+02, 1.00000000e-01, ...,
        1.00003797e-01, 1.00000000e-01, 1.00007585e-01],
       [1.76596715e+01, 3.29493914e+02, 1.00000000e-01, ...,
        1.00000445e-01, 1.00000000e-01, 1.00000974e-01],
       ...,
       [1.00046952e-01, 1.07809589e+02, 1.00000000e-01, ...,
        1.00004191e-01, 1.00000000e-01, 1.00000000e-01],
       [7.64092104e+00, 2.69495574e+02, 1.00000000e-01, ...,
        3.36377640e+00, 1.00180715e-01, 1.41864750e-01],
       [1.47374268e+00, 1.72279301e+03, 1.00002866e-01, ...,
        1.00012759e-01, 1.00001401e-01, 1.00000000e-01]])

In [37]:
type(lda.components_)

numpy.ndarray

In [38]:
lda.components_.shape

(10, 54777)

#### First topic

In [39]:
first_topic = lda.components_[0]

In [40]:
first_topic.shape

(54777,)

In [43]:
top_10_words = first_topic.argsort()[-10:]

In [47]:
for i in top_10_words:
    print(cv.get_feature_names()[i])

just
time
years
year
000
new
people
water
said
says


#### Second topic

In [48]:
second_topic = lda.components_[1]

In [51]:
top_20_words = second_topic.argsort()[-20:]

In [52]:
for i in top_20_words:
    print(cv.get_feature_names()[i])

rights
time
men
officers
country
violence
man
years
according
attack
black
say
reports
killed
told
city
says
police
people
said


# Printing the top 15 words in each topic

In [63]:
num_words = 15
lol = []
for i in range(len(lda.components_)):
    temp_list = []
    topic = lda.components_[i]
    top_15_words = topic.argsort()[-num_words:]
    for j in top_15_words:
        temp_list.append(cv.get_feature_names()[j])
    print(f"Topic {i+1}")
    print(temp_list)
    print("\n")

Topic 1
['company', 'national', 'oil', 'according', 'day', 'just', 'time', 'years', 'year', '000', 'new', 'people', 'water', 'said', 'says']


Topic 2
['violence', 'man', 'years', 'according', 'attack', 'black', 'say', 'reports', 'killed', 'told', 'city', 'says', 'police', 'people', 'said']


Topic 3
['says', 'state', 'new', 'npr', 'department', 'white', 'administration', 'security', 'obama', 'government', 'russia', 'house', 'president', 'said', 'trump']


Topic 4
['sanders', 'donald', 'republicans', 'vote', 'election', 'party', 'voters', 'state', 'republican', 'president', 'people', 'campaign', 'clinton', 'said', 'trump']


Topic 5
['don', 'just', 'percent', 'drug', 'women', 'like', 'children', 'disease', 'medical', 'patients', 'study', 'care', 'people', 'health', 'says']


Topic 6
['things', 've', 'music', 'new', 'life', 'don', 'way', 'time', 'really', 'know', 'think', 'people', 'says', 'just', 'like']


Topic 7
['series', 'people', 'film', 'best', 'years', 'game', 'says', 'team', 'w

### Conclusions

According to my observations:
    
    Topic 1 - Business News
    Topic 2 - Crime News
    Topic 3 - Political International News
    Topic 4 - Political National News
    Topic 5 - Health News
    Topic 6 - Lifestyle
    Topic 7 - Entertainment News
    Topic 8 - Judiciary News
    Topic 9 - Technology News
    Topic 10 - Education

In [64]:
topic_results = lda.transform(dtm)

In [65]:
topic_results.shape

(11992, 10)

In [69]:
topic_results[0].round(2).argmax()

2

In [79]:
npr['topic'] = topic_results.argmax(axis=1) + 1

In [80]:
npr["topic"].value_counts()

6     2278
2     1441
3     1406
10    1329
4     1299
5     1208
9     1054
7      893
1      795
8      289
Name: topic, dtype: int64

In [81]:
npr.head()

,Article,topic
0,"In the Washington of 2016, even when the polic...",3
1,Donald Trump has used Twitter — his prefe...,3
2,Donald Trump is unabashedly praising Russian...,3
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",10


In [86]:
npr['topic'].unique()

array([ 3, 10,  5,  6,  9,  1,  7,  2,  4,  8], dtype=int64)

In [ ]:
According to my observations:
    
    Topic 1 : Business News
    Topic 2 : Crime News
    Topic 3 : Political International News
    Topic 4 : Political National News
    Topic 5 : Health News
    Topic 6 : Lifestyle
    Topic 7 : Entertainment News
    Topic 8 : Judiciary News
    Topic 9 : Technology News
    Topic 10 : Education News

In [87]:
label = {1: "Business News", 
         2 : "Crime News",
         3 : "Political International News",
         4 : "Political National News",
         5 : "Health News",
         6 : "Lifestyle",
         7 : "Entertainment News",
         8 : "Judiciary News",
         9 : "Technology News",
         10 : "Education News",
         
        }

In [88]:
npr['topic_label'] = npr['topic'].apply(lambda n: label[n])

In [89]:
npr.head()

,Article,topic,topic_label
0,"In the Washington of 2016, even when the polic...",3,Political International News
1,Donald Trump has used Twitter — his prefe...,3,Political International News
2,Donald Trump is unabashedly praising Russian...,3,Political International News
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3,Political International News
4,"From photography, illustration and video, to d...",10,Education News


In [91]:
npr.iloc[1222]

Article        In the early 1990s, Seattle stood at the cente...
topic                                                          6
topic_label                                            Lifestyle
Name: 1222, dtype: object